In [33]:
import json
import time

In [78]:
class Base:
  def success(self, message, data=None):
    if data == None:
      return json.dumps({"status": "success", "message": message}, ensure_ascii=False)
    else:
      return json.dumps({"status": "success", "message": message, "data": data}, ensure_ascii=False)

  def error(self, message):
    return json.dumps({"status": "error", "message": message}, ensure_ascii=False)

In [79]:
class ShoppingCart(Base):
  def __init__(self):
    self.shopping_carts = []

  def add_cart(self, member_id):
    self.cart_id = len(self.shopping_carts) + 1
    self.member_id = member_id
    self.create_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
    data = {
      'cart_id': self.cart_id,
      'member_id': self.member_id,
      'create_time': self.create_time
    }
    self.shopping_carts.append(data)

  def get_carts(self):
    print(self.success("", self.shopping_carts))

# 測試
shopping_cart_instance = ShoppingCart()
shopping_cart_instance.add_cart(1)
shopping_cart_instance.add_cart(2)
shopping_cart_instance.add_cart(3)
shopping_cart_instance.get_carts()

{"status": "success", "message": "", "data": [{"cart_id": 1, "member_id": 1, "create_time": "2024-12-09 08:09:24"}, {"cart_id": 2, "member_id": 2, "create_time": "2024-12-09 08:09:24"}, {"cart_id": 3, "member_id": 3, "create_time": "2024-12-09 08:09:24"}]}


In [84]:
class Product(Base):
  def __init__(self):
    self.products = []

  def add_product(self, product_serialize):
    try:
      self.product_id = len(self.products) + 1
      self.create_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
      product = json.loads(product_serialize)
      product['product_id'] = self.product_id
      product['create_time'] = self.create_time
      product['last_update_time'] = self.create_time
      self.products.append(product)
      print(self.success("產品新增成功", product))
    except json.JSONDecodeError:
      print(self.error("產品資料格式不正確"))

  def get_product(self, product_id, output=True):
    for product in self.products:
      if product['product_id'] == product_id:
        if output: print(self.success("", product))
        return product
    if output: print(self.error(f"沒找到產品編號 {product_id}"))
    return {}

# 測試
product_instance = Product()

# 新增產品測試
product_instance.add_product('{"product_name": "電視機", "product_price": 50000, "product_init_qty": 10, "product_remaining_qty": 10, "product_status": "active", "product_screenshot_url": "/project/電視機.png"}')
product_instance.add_product('{"product_name": "冷氣", "product_price": 120000, "product_init_qty": 10, "product_remaining_qty": 10, "product_status": "active", "product_screenshot_url": "/project/冷氣.png"}')

# 取得產品測試
product_instance.get_product(1)
product_instance.get_product(3)


{"status": "success", "message": "產品新增成功", "data": {"product_name": "電視機", "product_price": 50000, "product_init_qty": 10, "product_remaining_qty": 10, "product_status": "active", "product_screenshot_url": "/project/電視機.png", "product_id": 1, "create_time": "2024-12-09 08:13:47", "last_update_time": "2024-12-09 08:13:47"}}
{"status": "success", "message": "產品新增成功", "data": {"product_name": "冷氣", "product_price": 120000, "product_init_qty": 10, "product_remaining_qty": 10, "product_status": "active", "product_screenshot_url": "/project/冷氣.png", "product_id": 2, "create_time": "2024-12-09 08:13:47", "last_update_time": "2024-12-09 08:13:47"}}
{"status": "success", "message": "", "data": {"product_name": "電視機", "product_price": 50000, "product_init_qty": 10, "product_remaining_qty": 10, "product_status": "active", "product_screenshot_url": "/project/電視機.png", "product_id": 1, "create_time": "2024-12-09 08:13:47", "last_update_time": "2024-12-09 08:13:47"}}
{"status": "error", "message": "

{}

In [104]:
class ShoppingCartItem(Base):
  def __init__(self, product_instance):
    self.cart_items = {}
    self.cart_item_id = 0
    self.product_instance = product_instance

  def add_cart_item(self, cart_id, product_id, quantity=1):
    # 初始化該購物車的商品列表
    if cart_id not in self.cart_items:
        self.cart_items[cart_id] = []

    # 檢查購物車內是否已有相同產品
    for item in self.cart_items[cart_id]:
      if item['product_id'] == product_id:
        print(self.error("產品已存在於購物車"))
        return

    # 從 Product 實例取得產品資料
    product = self.product_instance.get_product(product_id, False)

    # 如果產品尚未加入購物車
    self.cart_item_id += 1
    self.create_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
    new_item = {
      'cart_id': cart_id,
      'product_id': product_id,
      'cart_item_id': self.cart_item_id,
      'cart_item_serialize': product,
      'cart_item_price': product["product_price"],
      'cart_item_qty': quantity,
      'cart_item_screenshot_url': product["product_screenshot_url"],
      'cart_item_status': product["product_status"],
      'create_time': self.create_time,
      'last_update_time': self.create_time
    }
    self.cart_items[cart_id].append(new_item)
    print(self.success("產品已加入購物車", new_item))

  def remove_cart_item(self, cart_id, product_id):
    if cart_id in self.cart_items:
      filtered_items = [
        item for item in self.cart_items[cart_id] if item['product_id'] != product_id
      ]
      if len(filtered_items) < len(self.cart_items[cart_id]):
        self.cart_items[cart_id] = filtered_items
        print(self.success(f"產品編號 {product_id} 已從購物車編號 {cart_id} 移除"))
      else:
        print(self.error(f"購物車編號 {cart_id} 中找不到產品編號 {product_id}"))
    else:
      print(self.error(f"找不到購物車編號 {cart_id}"))

  def get_cart_item(self, cart_item_id):
    for cart_id, items in self.cart_items.items():
      for item in items:
        if item['cart_item_id'] == cart_item_id:
          print(self.success("找到購物車項目", item))
          return
    print(self.error(f"找不到購物車項目 {cart_item_id}"))

  def sync_cart_item_status(self, cart_id):
    if cart_id not in self.cart_items:
      print(self.error(f"找不到購物車編號 {cart_id}"))
      return

    updated_items = []
    for item in self.cart_items[cart_id]:
      # 取得最新的產品資料
      product = None
      for prod in self.product_instance.products:
        if prod["product_id"] == item["product_id"]:
          product = prod
          break

      item["cart_item_status"] = product.get("product_status")
      item["last_update_time"] = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
      updated_items.append(item)

    # 更新購物車的項目列表
    self.cart_items[cart_id] = updated_items
    print(self.success(f"購物車編號 {cart_id} 的所有商品狀態已同步", updated_items))

  def update_cart_item_qty(self, cart_item_id, cart_item_qty):
    # 確保數量有效
    if cart_item_qty <= 0:
      print(self.error("數量必須大於零"))
      return

    # 遍歷購物車查找目標項目
    for cart_id, items in self.cart_items.items():
      for item in items:
        if item['cart_item_id'] == cart_item_id:
          # 更新數量與最後更新時間
          item['cart_item_qty'] = cart_item_qty
          item['last_update_time'] = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
          print(self.success(f"購物車項目 {cart_item_id} 數量已更新為 {cart_item_qty}", item))
          return

    # 若找不到項目
    print(self.error(f"找不到購物車項目 {cart_item_id}"))

  def get_cart_items(self, cart_id):
    print(self.success("取得所有購物車項目", self.cart_items))

  def empty_cart_items(self, cart_id):
    if cart_id in self.cart_items:
      self.cart_items[cart_id] = []
      print(self.success(f"購物車編號 {cart_id} 的所有項目已清空"))
    else:
      print(self.error(f"找不到購物車編號 {cart_id}"))

# 測試
shopping_cart_item_instance = ShoppingCartItem(product_instance)
shopping_cart_item_instance.add_cart_item(shopping_cart_instance.shopping_carts[0]["cart_id"], product_instance.products[0]["product_id"], 1)
shopping_cart_item_instance.add_cart_item(shopping_cart_instance.shopping_carts[0]["cart_id"], product_instance.products[0]["product_id"], 2)
shopping_cart_item_instance.add_cart_item(shopping_cart_instance.shopping_carts[0]["cart_id"], product_instance.products[0]["product_id"]+1, 1)
time.sleep(1)
shopping_cart_item_instance.update_cart_item_qty(shopping_cart_item_instance.cart_items[shopping_cart_instance.shopping_carts[0]["cart_id"]][0]["cart_item_id"], 3)
shopping_cart_item_instance.get_cart_item(1)
shopping_cart_item_instance.sync_cart_item_status(1)
shopping_cart_item_instance.get_cart_items(shopping_cart_instance.shopping_carts[0]["cart_id"])
shopping_cart_item_instance.empty_cart_items(shopping_cart_instance.shopping_carts[0]["cart_id"])
shopping_cart_item_instance.get_cart_items(shopping_cart_instance.shopping_carts[0]["cart_id"])


{"status": "success", "message": "產品已加入購物車", "data": {"cart_id": 1, "product_id": 1, "cart_item_id": 1, "cart_item_serialize": {"product_name": "電視機", "product_price": 50000, "product_init_qty": 10, "product_remaining_qty": 10, "product_status": "active", "product_screenshot_url": "/project/電視機.png", "product_id": 1, "create_time": "2024-12-09 08:13:47", "last_update_time": "2024-12-09 08:13:47"}, "cart_item_price": 50000, "cart_item_qty": 1, "cart_item_screenshot_url": "/project/電視機.png", "cart_item_status": "active", "create_time": "2024-12-09 08:35:20", "last_update_time": "2024-12-09 08:35:20"}}
{"status": "error", "message": "產品已存在於購物車"}
{"status": "success", "message": "產品已加入購物車", "data": {"cart_id": 1, "product_id": 2, "cart_item_id": 2, "cart_item_serialize": {"product_name": "冷氣", "product_price": 120000, "product_init_qty": 10, "product_remaining_qty": 10, "product_status": "active", "product_screenshot_url": "/project/冷氣.png", "product_id": 2, "create_time": "2024-12-09 08:1